# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [31]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import pickle


import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import GridSearchCV

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///disasterresponses.db')
df = pd.read_sql_table('disasterresponses', engine)
X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize and remove stop words
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
     ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
# Split data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

In [6]:
# train classifier
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
# predict on test data
Y_pred = pipeline.predict(X_test)

In [8]:
def display_results(y_test, y_pred, category_names):
    
    metrics = []
    
    sum_accuracy = sum_f1 = sum_precision =sum_recall = 0
    
    # Report the f1 score, precision and recall for each output category of the dataset
    for i in range(len(category_names)):
        accuracy = accuracy_score(y_test[:, i], y_pred[:, i])
        f1 = f1_score(y_test[:, i], y_pred[:, i], average='weighted')
        precision = precision_score(y_test[:, i], y_pred[:, i], average='weighted')
        recall = recall_score(y_test[:, i], y_pred[:, i],average='weighted')
        metrics.append([accuracy, f1, precision, recall])
                
    # Format the results in a dataframe
    results = pd.DataFrame(metrics, index = category_names, columns = ['Accuracy','F1 score', 'Precision', 'Recall'])
    
    return results

In [9]:
# Define category names
category_names = Y.columns.values

In [10]:
# Show metrics results
results_RFC = display_results(np.array(Y_test), Y_pred, category_names)
results_RFC

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


,Accuracy,F1 score,Precision,Recall
related,0.806225,0.795891,0.794991,0.806225
request,0.884040,0.870079,0.875890,0.884040
offer,0.996186,0.994282,0.992386,0.996186
aid_related,0.753128,0.747564,0.753254,0.753128
medical_help,0.925999,0.905795,0.905734,0.925999
medical_products,0.955294,0.941132,0.947761,0.955294
search_and_rescue,0.970705,0.959016,0.958314,0.970705
security,0.982301,0.973833,0.965509,0.982301
military,0.966585,0.954639,0.955129,0.966585
water,0.958041,0.951089,0.953702,0.958041


In [11]:
# Show metrics average
results_RFC.mean()

Accuracy     0.942744
F1 score     0.930316
Precision    0.931768
Recall       0.942744
dtype: float64

### 6. Improve your model
Use grid search to find better parameters. 

In [12]:
parameters = {
    'vect__ngram_range': ((1, 1), (1, 2), (2, 2)),
     #'tfidf__use_idf': (True, False),
    'clf__estimator__n_estimators':[50, 100]
}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=10)

In [13]:
cv.fit(X_train, Y_train)

Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV] clf__estimator__n_estimators=50, vect__ngram_range=(1, 1) .......
[CV]  clf__estimator__n_estimators=50, vect__ngram_range=(1, 1), score=0.2525175465364663, total= 3.3min
[CV] clf__estimator__n_estimators=50, vect__ngram_range=(1, 1) .......


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.7min remaining:    0.0s


[CV]  clf__estimator__n_estimators=50, vect__ngram_range=(1, 1), score=0.2580103753433018, total= 3.3min
[CV] clf__estimator__n_estimators=50, vect__ngram_range=(1, 1) .......


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  7.4min remaining:    0.0s


[CV]  clf__estimator__n_estimators=50, vect__ngram_range=(1, 1), score=0.2545010680500458, total= 3.3min
[CV] clf__estimator__n_estimators=50, vect__ngram_range=(1, 2) .......


...

[CV]  clf__estimator__n_estimators=50, vect__ngram_range=(2, 2), score=0.23176685993286542, total= 9.4min
[CV] clf__estimator__n_estimators=100, vect__ngram_range=(1, 1) ......


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 67.2min remaining:    0.0s


[CV]  clf__estimator__n_estimators=100, vect__ngram_range=(1, 1), score=0.25556911809581934, total= 6.3min
[CV] clf__estimator__n_estimators=100, vect__ngram_range=(1, 1) ......
[CV]  clf__estimator__n_estimators=100, vect__ngram_range=(1, 1), score=0.2622825755263961, total= 6.3min
[CV] clf__estimator__n_estimators=100, vect__ngram_range=(1, 1) ......
[CV]  clf__estimator__n_estimators=100, vect__ngram_range=(1, 1), score=0.26731766859932865, total= 6.3min
[CV] clf__estimator__n_estimators=100, vect__ngram_range=(1, 2) ......
[CV]  clf__estimator__n_estimators=100, vect__ngram_range=(1, 2), score=0.25785779676533416, total=13.8min
[CV] clf__estimator__n_estimators=100, vect__ngram_range=(1, 2) ......
[CV]  clf__estimator__n_estimators=100, vect__ngram_range=(1, 2), score=0.2679279829111993, total=13.5min
[CV] clf__estimator__n_estimators=100, vect__ngram_range=(1, 2) ......
[CV]  clf__estimator__n_estimators=100, vect__ngram_range=(1, 2), score=0.26533414708574915, total=14.3min
[CV] 

[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed: 200.3min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'vect__ngram_range': ((1, 1), (1, 2), (2, 2)), 'clf__estimator__n_estimators': [50, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=10)

In [14]:
cv.best_params_

{'clf__estimator__n_estimators': 100, 'vect__ngram_range': (1, 2)}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [15]:
# predict on test data based on GridSearch Parameter
Y_pred_tuned = cv.predict(X_test)

In [17]:
# Show metrics results
results_RFC_tuned = display_results(np.array(Y_test), Y_pred_tuned, category_names)
results_RFC_tuned

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


,Accuracy,F1 score,Precision,Recall
related,0.815532,0.801765,0.803101,0.815532
request,0.898230,0.886603,0.894374,0.898230
offer,0.996186,0.994282,0.992386,0.996186
aid_related,0.773268,0.769024,0.773527,0.773268
medical_help,0.923863,0.893789,0.898480,0.923863
medical_products,0.952243,0.933728,0.942239,0.952243
search_and_rescue,0.972383,0.961247,0.967541,0.972383
security,0.982148,0.974047,0.969130,0.982148
military,0.966128,0.951193,0.953723,0.966128
water,0.960177,0.953046,0.957580,0.960177


In [18]:
# Show metrics average
results_RFC_tuned.mean()

Accuracy     0.945416
F1 score     0.932625
Precision    0.935009
Recall       0.945416
dtype: float64

In [19]:
# Compare model metrics
compare_model = pd.DataFrame(index=['Accuracy','F1 score', 'Precision', 'Recall'])
compare_model['RFC'] = results_RFC.mean()
compare_model['RFC_tuned'] = results_RFC_tuned.mean()
compare_model

,RFC,RFC_tuned
Accuracy,0.942744,0.945416
F1 score,0.930316,0.932625
Precision,0.931768,0.935009
Recall,0.942744,0.945416


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

We will test our model against multinomial logistic regression and SVM to see which one would fit better our dataset. 

#### 8.1. Multinomial logistic regression

In [20]:
pipeline_MLR = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(LogisticRegression(multi_class='multinomial', solver='newton-cg')))
    ])

In [21]:
# train classifier
pipeline_MLR.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...None, solver='newton-cg',
          tol=0.0001, verbose=0, warm_start=False),
           n_jobs=1))])

In [22]:
# predict on test data
Y_pred_MLR = pipeline_MLR.predict(X_test)

In [23]:
# Show metrics results
results_MLR = display_results(np.array(Y_test), Y_pred_MLR, category_names)
results_MLR

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


,Accuracy,F1 score,Precision,Recall
related,0.827586,0.810323,0.811480,0.827586
request,0.900519,0.892602,0.894685,0.900519
offer,0.996186,0.994282,0.992386,0.996186
aid_related,0.779371,0.777193,0.778186,0.779371
medical_help,0.925694,0.909798,0.906968,0.925694
medical_products,0.955905,0.943964,0.947267,0.955905
search_and_rescue,0.972841,0.961773,0.970606,0.972841
security,0.982606,0.973985,0.965515,0.982606
military,0.968569,0.959521,0.960560,0.968569
water,0.962008,0.958304,0.958215,0.962008


In [24]:
results_MLR.mean()

Accuracy     0.948520
F1 score     0.939608
Precision    0.940451
Recall       0.948520
dtype: float64

#### 8.2. Support Vector Machine

In [25]:
pipeline_SVC = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(SVC()))
    ])

In [26]:
# train classifier
pipeline_SVC.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...bility=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
           n_jobs=1))])

In [27]:
# predict on test data
Y_pred_SVC = pipeline_SVC.predict(X_test)

In [28]:
# Show metrics result
results_SVC = display_results(np.array(Y_test), Y_pred_SVC, category_names)
results_SVC

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


,Accuracy,F1 score,Precision,Recall
related,0.752212,0.645839,0.565823,0.752212
request,0.829722,0.752507,0.688439,0.829722
offer,0.996186,0.994282,0.992386,0.996186
aid_related,0.584376,0.431079,0.341495,0.584376
medical_help,0.922948,0.885965,0.851833,0.922948
medical_products,0.949649,0.925124,0.901833,0.949649
search_and_rescue,0.970705,0.956275,0.942268,0.970705
security,0.982606,0.973985,0.965515,0.982606
military,0.965822,0.949031,0.932813,0.965822
water,0.938816,0.909189,0.881375,0.938816


In [29]:
# Compare model metrics
compare_model['MLR'] = results_MLR.mean()
compare_model['SVC'] = results_SVC.mean()
compare_model

,RFC,RFC_tuned,MLR,SVC
Accuracy,0.942744,0.945416,0.948520,0.923789
F1 score,0.930316,0.932625,0.939608,0.889645
Precision,0.931768,0.935009,0.940451,0.861194
Recall,0.942744,0.945416,0.948520,0.923789


Based on the model metrics, the multinomial logistic regression model seems to perform better than the random forest classifier or the support vector machine models. Therefore we choose the multinomial logistic regression as model.

### 9. Export your model as a pickle file

In [32]:
pickle.dump(pipeline_MLR, open('model.sav', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.